In [1]:
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from scipy.stats import pearsonr
import tensorflow as tf
import torch
import torchvision
sys.path.append('/home/juan/Documents/Master_Project/code/my_work/Testing-Algorithms/')
from rf_improved2_copy import TorchRandomForestRegressor
from tqdm import tqdm
from math import sqrt
import warnings
from sklearn.exceptions import DataConversionWarning


2024-03-27 13:58:51.013191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 13:58:51.123323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 13:58:51.986216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:


data1 =pd.read_csv('data1.csv')
data9 = pd.read_csv('data_1_test.csv')

df_1 = data1.iloc[:, 1:]
df_9 = data9.iloc[:,1:]

columns = df_1.columns.tolist()

columns.remove("Folder")
columns.remove("QM/MM SP Barrier")

new_columns_order = columns + ["Folder", "QM/MM SP Barrier"]

df_1 = df_1.reindex(columns=new_columns_order)
df_1.head()

,661,666,156,444,705,407,807,650,356,699,...,Box_26,Box_33,Box_34,Box_46,Box_50,Box_51,Total Charge,internal_vdw_energy,Folder,QM/MM SP Barrier
0,0,0,1,1,0,0,1,1,2,0,...,-0.241552,0.0,0.0,0.000000,-0.758976,-1.007604,1.000009,2.925351,mol_134810,29.71
1,0,0,0,0,0,1,0,0,5,0,...,0.000000,0.0,0.0,-0.056827,-0.020914,-0.183128,-0.000017,-4.432160,mol_147462,28.33
2,0,0,0,0,0,0,1,2,4,0,...,-1.176918,0.0,0.0,-0.249862,-0.349354,0.000000,0.000008,-5.104365,mol_22419,28.28
3,0,0,0,0,0,1,1,1,2,0,...,0.096336,0.0,0.0,0.000000,-0.883456,0.819872,-0.000009,-3.150597,mol_469,22.72
4,1,0,0,0,0,0,1,1,3,1,...,0.026424,0.0,0.0,0.000000,0.662524,-1.430314,0.999997,-2.510429,mol_67286,25.81


In [3]:
df = pd.read_csv('/home/juan/Documents/Master_Project/data/process/final_selection/All_feats_newC_hp_atoms.csv')
df = df.iloc[:,1:]
df.head()

,nBondsD,nBonds2,nBondsT,nBondsM,pubchemcomplexity,pubchemhbondacceptor,pubchemhbonddonor,pubchemrotbon,pubchemlogp,pubchemmass,...,hp_Atom_31,hp_Atom_32,hp_Atom_33,hp_Atom_34,hp_Atom_35,hp_Atom_36,hp_Atom_37,hp_Atom_38,Folder,QM/MM SP Barrier
0,1,11,0,11,0.39820,5,2,4,0.39820,285.371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_134810,29.71
1,0,14,0,12,3.35790,4,0,3,3.35790,309.413,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_147462,28.33
2,2,14,0,11,1.79150,6,1,7,1.79150,340.427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_22419,28.28
3,1,16,1,6,1.46888,6,0,6,1.46888,316.405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_469,22.72
4,1,12,0,6,-0.04998,4,3,2,-0.04998,252.313,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mol_67286,25.81


In [4]:
import pandas as pd

dataset_initial = pd.read_csv('dataset.csv')

df00 = dataset_initial.iloc[:, 1:]

df00.drop(columns='total_charge', inplace=True)

df00.tail()


,0,1,2,3,4,5,6,7,8,9,...,resid 120,resid 119,resid 146,resid 145,resid 147,resid 786,resid 61,resid 29,Folder,QM/MM SP Barrier
1092,0,0,0,0,0,0,0,0,0,0,...,19.635709,22.622308,19.836394,20.933459,22.265739,15.378876,5.409570,20.505763,mol_178096,21.45
1093,0,0,0,0,0,0,0,0,0,0,...,18.754397,22.329591,19.444639,20.593992,21.991518,15.392180,4.830849,20.268452,mol_178159,27.00
1094,0,0,0,0,0,0,0,0,0,0,...,17.451327,21.067321,18.741670,19.913888,20.867617,13.842437,3.824708,18.920998,mol_178160,29.97
1095,0,0,0,0,0,0,0,0,0,0,...,19.142024,22.027824,19.150536,20.300864,21.613279,15.685708,5.110045,19.831135,mol_178411,28.04
1096,0,0,0,0,0,0,0,0,0,0,...,19.653265,22.952895,20.551297,21.449449,22.798516,16.306594,5.876727,21.242883,mol_178649,25.78


## Common functions

In [5]:
def preprocess_data_manual_no_split(df, use_torch):
    X = df.drop(columns=['QM/MM SP Barrier', 'Folder']).values
    Y = df['QM/MM SP Barrier'].values
    ss = StandardScaler()
    Y_reshaped = Y.reshape(-1, 1)
    Y_normalized = ss.fit_transform(Y_reshaped)
    if use_torch:
        X = torch.tensor(X, dtype=torch.float32)
        Y_normalized = torch.tensor(Y_normalized, dtype=torch.float32).flatten()
    return X, Y_normalized, ss

def mean_deviation_from_true_mean(y_true, y_pred):
    """
    Calculates the mean of the absolute deviations of predicted values from the mean of true values.

    :param y_true: array-like, true values.
    :param y_pred: array-like, predicted values.
    :return: float, mean deviation from the true mean.
    """
    true_mean = np.mean(y_true)
    deviations = np.abs(y_pred - true_mean)
    mean_deviation = np.mean(deviations)
    return mean_deviation

# RF (sklearn)

In [6]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import warnings
from tqdm import tqdm

def cross_validate_with_blind_set(X, y, X_blind, y_blind=None, n_splits=10, use_torch=False):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores, mae_scores, variance_scores, r2_scores, pearson_corr_scores, deviations = [], [], [], [], [], []
    
    for train_index, test_index in tqdm(kf.split(X), total=n_splits, desc="Cross-Validation"):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Model initialization
        if use_torch:
            model = TorchRandomForestRegressor(nb_trees=8, nb_samples=40, max_depth=200, bootstrap=True)
        else:
            model = RandomForestRegressor(
                n_estimators=160,
                criterion="squared_error",
                max_depth=30,
                min_samples_split=2,
                min_samples_leaf=1,
                max_features=0.8,
                bootstrap=True,
                oob_score=True,
                n_jobs=-1,
                random_state=40,
                ccp_alpha=0.00,
                max_samples=None,
                monotonic_cst=None)
        
        # Fit the model
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test).flatten()
        y_test = y_test.flatten()  # Ensure y_test is flattened

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        variance = np.var(y_test - y_pred)
        r2 = r2_score(y_test, y_pred)
        pearson_corr = pearsonr(y_test, y_pred)[0]  # Ensure both are flattened
        deviation = mean_deviation_from_true_mean(y_test, y_pred)
        
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        variance_scores.append(variance)
        r2_scores.append(r2)
        pearson_corr_scores.append(pearson_corr)
        deviations.append(deviation)

    # Calculate mean of collected scores
    mean_rmse = np.mean(rmse_scores)
    mean_mae = np.mean(mae_scores)
    mean_variance = np.mean(variance_scores)
    mean_r2 = np.mean(r2_scores)
    mean_pearson_corr = np.mean(pearson_corr_scores)
    mean_deviation = np.mean(deviations)


    return mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae

def complete_analysis_pipeline(df, title, use_torch=False):
    X, Y_normalized, ss = preprocess_data_manual_no_split(df, use_torch)
    train_percentage = 0.99
    n_samples = len(X)
    n_train_samples = int(train_percentage * n_samples)
    X_train, Y_train = X[:n_train_samples], Y_normalized[:n_train_samples]
    X_blind, Y_blind = X[n_train_samples:], Y_normalized[n_train_samples:]

    mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae = cross_validate_with_blind_set(X_train, Y_train, X_blind, Y_blind, use_torch=use_torch)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)


In [7]:
complete_analysis_pipeline(df_1, ' ', use_torch=False)

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Cross-Validation: 100%|██████████| 10/10 [00:40<00:00,  4.07s/it]

0.9182877574361046 0.4101920920534986 0.41547360002274936 0.15597504650369887 0.8441062947325942 0.674351020676687


In [8]:
complete_analysis_pipeline(df00, ' ', use_torch=False)

Cross-Validation: 100%|██████████| 10/10 [00:05<00:00,  1.67it/s]

0.9782638619032505 0.2646721537406801 0.3303770430952365 0.036489566116692504 0.9507574306334888 0.7587205851508054


In [9]:
complete_analysis_pipeline(df_9, ' ', use_torch=False)

Cross-Validation: 100%|██████████| 10/10 [00:26<00:00,  2.63s/it]

0.9183418115336476 0.4108413030166854 0.42115689104229437 0.1557974904990413 0.8439591113517139 0.675380018316474


In [10]:
complete_analysis_pipeline(df, ' ', use_torch=False)

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Cross-Validation: 100%|██████████| 10/10 [01:30<00:00,  9.08s/it]

0.9152752316701157 0.41351426977965067 0.4112832718570559 0.16158221913484316 0.838585899656471 0.6710236740221227


# SVR

In [11]:
from sklearn.svm import SVR

def cross_validate_with_blind_set(X, y, X_blind, y_blind=None, n_splits=10, use_torch=False):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores, mae_scores, variance_scores, r2_scores, pearson_corr_scores, deviations = [], [], [], [], [], []

    for train_index, test_index in tqdm(kf.split(X), total=n_splits, desc="Cross-Validation"):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        if use_torch:
            pass  # Future implementation
        else:
            model = SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=1e-3,
                        C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
        
        # Fit the model
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test).flatten()
        y_test = y_test.flatten() 

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        variance = np.var(y_test - y_pred)
        r2 = r2_score(y_test, y_pred)
        pearson_corr = pearsonr(y_test, y_pred)[0]
        deviation = mean_deviation_from_true_mean(y_test, y_pred)
        
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        variance_scores.append(variance)
        r2_scores.append(r2)
        pearson_corr_scores.append(pearson_corr)
        deviations.append(deviation)

    # Calculate and return mean of collected scores
    mean_rmse = np.mean(rmse_scores)
    mean_mae = np.mean(mae_scores)
    mean_variance = np.mean(variance_scores)
    mean_r2 = np.mean(r2_scores)
    mean_pearson_corr = np.mean(pearson_corr_scores)
    mean_deviation = np.mean(deviations)

    return mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae

def complete_analysis_pipeline(df, title, use_torch=False):
    X, Y_normalized, ss = preprocess_data_manual_no_split(df, use_torch)
    train_percentage = 0.99
    n_samples = len(X)
    n_train_samples = int(train_percentage * n_samples)
    X_train, Y_train = X[:n_train_samples], Y_normalized[:n_train_samples]
    X_blind, Y_blind = X[n_train_samples:], Y_normalized[n_train_samples:]

    mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae = cross_validate_with_blind_set(X_train, Y_train, X_blind, Y_blind, use_torch=use_torch)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)


In [12]:
complete_analysis_pipeline(df_1, ' ', use_torch=False)

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Cross-Validation: 100%|██████████| 10/10 [02:39<00:00, 15.98s/it]

0.9468217768802687 0.3396696347022214 0.20391694770903518 0.10317280443026053 0.8961143333931865 0.6919297969382504


In [13]:
complete_analysis_pipeline(df00, ' ', use_torch=False)

Cross-Validation: 100%|██████████| 10/10 [00:02<00:00,  4.45it/s]

0.9858394163849302 0.20898893618433986 0.13786038909511852 0.02136004445783358 0.960766035009782 0.7616443318093069


In [14]:
complete_analysis_pipeline(df_9, ' ', use_torch=False)

Cross-Validation: 100%|██████████| 10/10 [00:32<00:00,  3.27s/it]

0.9465200640611988 0.34098378693483933 0.20393515426207606 0.10373772811594469 0.8954720306212826 0.6917086287548841


In [15]:
complete_analysis_pipeline(df, ' ', use_torch=False)

Cross-Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Cross-Validation: 100%|██████████| 10/10 [04:25<00:00, 26.52s/it]

0.9470567115468193 0.3396587398804555 0.20175842575319325 0.10275457934503392 0.8965280943462794 0.6921915241243795


# Torch

In [16]:
def cross_validate_with_blind_set(X, y, X_blind, y_blind=None, n_splits=10, use_torch=False):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics_results = {
        "rmse_scores": [],
        "mae_scores": [],
        "variance_scores": [],
        "r2_scores": [],
        "pearson_corr_scores": [],
        "deviations": []
    }

    # StandardScaler instance
    ss = StandardScaler()

    for train_index, test_index in tqdm(kf.split(X), total=n_splits, desc="Cross-Validation"):
        if use_torch:
            # To torch tensors
            X_train, X_test = torch.tensor(X[train_index], dtype=torch.float32), torch.tensor(X[test_index], dtype=torch.float32)
            y_train, y_test = torch.tensor(y[train_index], dtype=torch.float32).view(-1, 1), torch.tensor(y[test_index], dtype=torch.float32).view(-1, 1)
            
            # Standardize target values
            y_train_np = ss.fit_transform(y_train.numpy())
            y_test_np = ss.transform(y_test.numpy())
            y_train, y_test = torch.from_numpy(y_train_np), torch.from_numpy(y_test_np)

            model = TorchRandomForestRegressor(nb_trees=8, nb_samples=40, max_depth=100, bootstrap=True)
            model.fit(X_train, y_train.view(-1))
            
            # Predict and convert predictions to numpy for metrics calculation
            y_pred = [model.predict(torch.FloatTensor(test_vector)) for test_vector in X_test]
            y_pred_np = np.array([pred.numpy() for pred in y_pred])
            y_test = y_test.numpy().flatten()
            
    # If y_test is a PyTorch tensor, convert it to a NumPy array
    # Ensure y_test is detached from the computation graph and converted
    y_test_np = y_test   
    rmse = np.sqrt(mean_squared_error(y_test_np, y_pred_np))
    mae = mean_absolute_error(y_test_np, y_pred_np)
    variance = np.var(y_test_np - y_pred_np)
    r2 = r2_score(y_test_np, y_pred_np)
    pearson_corr, _ = pearsonr(y_test_np, y_pred_np)
    deviation = mean_deviation_from_true_mean(y_test_np, y_pred_np)

    metrics_results["rmse_scores"].append(rmse)
    metrics_results["mae_scores"].append(mae)
    metrics_results["variance_scores"].append(variance)
    metrics_results["r2_scores"].append(r2)
    metrics_results["pearson_corr_scores"].append(pearson_corr)
    metrics_results["deviations"].append(deviation)

    # After the loop, aggregate and return the results
    mean_rmse = np.mean(metrics_results["rmse_scores"])
    mean_mae = np.mean(metrics_results["mae_scores"])
    mean_variance = np.mean(metrics_results["variance_scores"])
    mean_r2 = np.mean(metrics_results["r2_scores"])
    mean_pearson_corr = np.mean(metrics_results["pearson_corr_scores"])
    mean_deviation = np.mean(metrics_results["deviations"])

    return mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae



def complete_analysis_pipeline(df, title, use_torch=False):
    X, Y_normalized, ss = preprocess_data_manual_no_split(df, use_torch)
    train_percentage = 0.99
    n_samples = len(X)
    n_train_samples = int(train_percentage * n_samples)
    X_train, Y_train = X[:n_train_samples], Y_normalized[:n_train_samples]
    X_blind, Y_blind = X[n_train_samples:], Y_normalized[n_train_samples:]

    mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae = cross_validate_with_blind_set(X_train, Y_train, X_blind, Y_blind, use_torch=use_torch)
    print(mean_rmse, mean_pearson_corr, mean_deviation, mean_r2, mean_variance, mean_mae)


In [ ]:
complete_analysis_pipeline(df_1, ' ', use_torch=True)

In [ ]:
complete_analysis_pipeline(df00, ' ', use_torch=True)

In [ ]:
complete_analysis_pipeline(df_9, ' ', use_torch=True)

In [ ]:
complete_analysis_pipeline(df, ' ', use_torch=True)